In [ ]:
%matplotlib inline 
import cv2
from __future__ import division
from pylab import *
import skimage as ski
from skimage import data, io, filters, exposure
from skimage.filters import rank
from skimage import img_as_float, img_as_ubyte
from skimage.morphology import disk
import skimage.morphology as mp
from skimage import util
from skimage.color import rgb2hsv, hsv2rgb, rgb2gray
from skimage.filters.edges import convolve
from matplotlib import pylab as plt
import numpy as np
from numpy import array
from IPython.display import display
from ipywidgets import interact, interactive, fixed
from ipywidgets import *
from ipykernel.pylab.backend_inline import flush_figures
import imutils
from pyimagesearch.shapedetector import ShapeDetector
import argparse
import imutils
import cv2
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import os


preprocessedB = []
preprocessedN = []
images = []

def find_blue(name, con):
    image = cv2.imread(name)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    maxx = int(con[2] / 5)
    for row in range (-maxx, maxx):
        i = con[0] + row
        if(i < 0):
            return False
        if(i>=len(image)):
            return False
        if(gray[i][con[1]+int(0.5*con[3])] != 0):
            return True
    return False

def key(plate, x):
    key = 0 
    kernel = np.ones((3,3),np.float32)
    cut = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    #cut = cv2.filter2D(cut,-1,kernel/9)
    cut = cv2.bilateralFilter(cut, 9, 75, 75)
    #cut = cv2.fastNlMeansDenoising(cut,None,10,7,21)
    #cut = cv2.equalizeHist(cut)
    #ret, cut = cv2.threshold(cut,254,255,cv2.THRESH_BINARY)
        
    #cut = cv2.morphologyEx(cut,cv2.MORPH_OPEN,np.ones((1,1)))
    cv2.imwrite("asdf/TESTOWO" + str(x) + ".png", cut)
    if(os.path.exists("asdf/TESTOWO" + str(x) + ".png") and os.path.getsize("asdf/TESTOWO" + str(x) + ".png")>0):
        im = Image.open("asdf/TESTOWO" + str(x) + ".png")
        #if im > 0:
        result=pytesseract.image_to_string(im)
            
        if(len(result)>0):
            key += len(result)
        #    cv2.imshow("asdf",cut)    
            #cv2.drawContours(original, [approx], -1, (0, 255, 0), 2)
            #cv2.putText(original, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
        #0.5, (255, 0, 0), 2)
            print(result)
            
        im.close()
    return key
    
def find(name, name2, original): 
    image = cv2.imread(name)
    original = np.asarray(original)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(gray.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[1]
    sd = ShapeDetector()
    plates = []
    for c in cnts:     
        M = cv2.moments(c)
        if M["m00"] == 0.0:            
            continue
        cX = int((M["m10"] / M["m00"]))# * ratio)
        cY = int((M["m01"] / M["m00"]))# * ratio)
        shape = sd.detect(c)
        if shape != "rectangle":
            continue
        #rows, cols = image.shape
        epsilon = 0.01*cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, epsilon, True)
        con = x, y, w, h = cv2.boundingRect(approx)
        
        rect = cv2.minAreaRect(approx)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        W = rect[1][0]
        H = rect[1][1]
        
        
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        
        angle = rect[2]
        
        if angle < -45:
            angle += 90
        
        if np.abs(angle) > 25:
            continue
      
        center = ((x1+x2)/2,(y1+y2)/2)
        
        size = (x2-x1, y2-y1)
        
        M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)
        cropped = cv2.getRectSubPix(image, size, center)
        cropped = cv2.warpAffine(cropped, M, size)
        croppedW = H if H > W else W
        croppedH = H if H < W else W
        if croppedW < 130 or croppedH < 25:
            continue
        if np.abs(croppedW/croppedH - 5) > 3.0 :
            continue  
        
        croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW),int(croppedH)), (size[0]/2, size[1]/2))
        
        if(key(croppedRotated, x) > 0):
        #rot = cv2.getRotationMatrix2D(rect[0], rect[2]-90, 1)
        #temp = cv2.warpAffine(image, rot, (rows,cols))
        #cut = image[y:y+h, x:x+w]
        
        #plates.append(image, [c, con )

        #if not find_blue(name2, con):
        #    continue
            cv2.drawContours(original, [box], -1, (0, 255, 0), 2)

    cv2.imshow("Image", original)
    cv2.waitKey(0)
def preprocess(name):
    kernel = np.ones((1,1),np.uint8)
    kernel2 = np.ones((4,4),np.uint8)

    zdjecie=cv2.imread("Samochody/" + name)
    resized = imutils.resize(zdjecie, width=1600)
    images.append(resized)    
    gray=cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
        
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    res = cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
        cv2.THRESH_BINARY,11,2)       
    hsv=cv2.cvtColor(resized,cv2.COLOR_BGR2HSV)
    
    low=np.array([0,0,30])
    high=np.array([255,90,254])
    maska=cv2.inRange(hsv,low,high)
    res = res & maska
    
    low2=np.array([100,90,20])
    high2=np.array([130,255,250])
    maska2=cv2.inRange(hsv,low2,high2)    
    
    res = cv2.morphologyEx(res,cv2.MORPH_OPEN,kernel)
    maska2 = cv2.morphologyEx(maska2,cv2.MORPH_OPEN,kernel2)
        
    cv2.imwrite("Samochody_filtr/B" + name, res )
    cv2.imwrite("Samochody_filtr/N" + name, maska2 )

def main():
    pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR/tesseract'
    tablice=["1", "cytryna", "merc", "test", "tunel"]
    for i, tablica in enumerate(tablice):
        tablice[i] = tablica + ".png"
    
    for i in range(1,17):
        tablice.append("sam" + str(i) + ".png")
        
    for tablica in tablice:
        print(tablica)
        preprocessedB.append("Samochody_filtr/B" + tablica)
        preprocessedN.append("Samochody_filtr/N" + tablica)
        preprocess(tablica)
    for preB, preN, image in zip(preprocessedB, preprocessedN, images):
        find(preB, preN, image)
    
if __name__=="__main__":
    main()

1.png
cytryna.png
merc.png
test.png
tunel.png
sam1.png
sam2.png
sam3.png
sam4.png
sam5.png
sam6.png
sam7.png
sam8.png
sam9.png
sam10.png
sam11.png
sam12.png
sam13.png
sam14.png
sam15.png
sam16.png
WIT/;s:s:«‘Vﬁ¥5?_-Z4,
WE-98LVf
‘ u:ua_zs_o2
DU<3\/B523
STA-3425Si
lr30_:6R68o4
\
IQN2 6109
!P_Q _6A729
l P_0.8J.9lLA
lLW~267,oaJ
K ‘- A143!
ISZré7.='_1.9912


In [8]:



image = cv2.imread("Samochody_filtr/Bauto01.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = gray
cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[1]
sd = ShapeDetector()
for c in cnts:     
    M = cv2.moments(c)
    if M["m00"] == 0.0:            
        continue
    cX = int((M["m10"] / M["m00"]))# * ratio)
    cY = int((M["m01"] / M["m00"]))# * ratio)
    shape = sd.detect(c)
    if shape != "rectangle":
        continue
    epsilon = 0.01*cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, epsilon, True)
    x, y, w, h = cv2.boundingRect(approx)
    #print str(x)+";"+str(y)+";"+str(w)+";"+str(h)+";"
    if w < 80 or h < 30:
        continue
    if w>0 and h>0:#x<891 and x>631 and y<720 and y>648:
        cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
        continue
        #cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 0, 0), 2)
        cv2.imwrite("asdf/TESTOWO" + str(x) + ".png",image[x:x+h, y:y+w])
        if(os.path.exists("asdf/TESTOWO" + str(x) + ".png") and os.path.getsize("asdf/TESTOWO" + str(x) + ".png")>0):
            img=Image.open("asdf/TESTOWO" + str(x) + ".png")
            result=pytesseract.image_to_string(img)
            if(len(result)>0):
                cv2.imshow("asdf",image[x:x+h, y:y+w])    
                cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
            print(result)
            img.close()
#cv2.imshow("asdf",image[631:631+100, 648:648+720-648])

cv2.imshow("Image", image)
cv2.waitKey(0)

error: D:\Build\OpenCV\opencv-3.3.1\modules\imgproc\src\color.cpp:11048: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor
